## CSCI 470 Activities and Case Studies

1. For all activities, you are allowed to collaborate with a partner. 
1. For case studies, you should work individually and are **not** allowed to collaborate.

By filling out this notebook and submitting it, you acknowledge that you are aware of the above policies and are agreeing to comply with them.

Some considerations with regard to how these notebooks will be graded:

1. You can add more notebook cells or edit existing notebook cells other than "# YOUR CODE HERE" to test out or debug your code. We actually highly recommend you do so to gain a better understanding of what is happening. However, during grading, **these changes are ignored**. 
2. You must ensure that all your code for the particular task is available in the cells that say "# YOUR CODE HERE"
3. Every cell that says "# YOUR CODE HERE" is followed by a "raise NotImplementedError". You need to remove that line. During grading, if an error occurs then you will not receive points for your work in that section.
4. If your code passes the "assert" statements, then no output will result. If your code fails the "assert" statements, you will get an "AssertionError". Getting an assertion error means you will not receive points for that particular task.
5. If you edit the "assert" statements to make your code pass, they will still fail when they are graded since the "assert" statements will revert to the original. Make sure you don't edit the assert statements.
6. We may sometimes have "hidden" tests for grading. This means that passing the visible "assert" statements is not sufficient. The "assert" statements are there as a guide but you need to make sure you understand what you're required to do and ensure that you are doing it correctly. Passing the visible tests is necessary but not sufficient to get the grade for that cell.
7. When you are asked to define a function, make sure you **don't** use any variables outside of the parameters passed to the function. You can think of the parameters being passed to the function as a hint. Make sure you're using all of those variables.
8. Finally, **make sure you run "Kernel > Restart and Run All"** and pass all the asserts before submitting. If you don't restart the kernel, there may be some code that you ran and deleted that is still being used and that was why your asserts were passing.

# Feature Learning

In this exercise we will run feature learning methods on a dataset to identify the key features to use for predicting a target within it.

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk

plt.style.use("ggplot")

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

The data we'll be working with is the [California housing dataset](http://scikit-learn.org/stable/datasets/index.html#california-housing-dataset).

In [3]:
house_data = fetch_california_housing()

In [4]:
print(house_data["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

By reading the above information about the data, which features do you intuit meaningful for predicting the target? Which are not meaningful?

In [5]:
house_features = pd.DataFrame(house_data["data"], columns=house_data["feature_names"])
house_prices = pd.Series(house_data["target"])

In [6]:
house_features.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [7]:
house_features.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [8]:
house_prices.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
dtype: float64

In [9]:
house_prices.describe()

count    20640.000000
mean         2.068558
std          1.153956
min          0.149990
25%          1.196000
50%          1.797000
75%          2.647250
max          5.000010
dtype: float64

First, we'll split our data in order to follow the process we had outlined several lectures ago for effectively evaluating supervised learning problems.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(house_features, house_prices, test_size=0.2, random_state=0)

## Filter Feature Selection

Select the best features based on [mutual information score](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression) from the training data then transform X_train and X_test into the new shape for the data. See [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)

In [11]:
# We will select the best k features from all feature selection methods
k = 4

In [12]:
# Save the transformed values into mi_X_train and mi_X_test
# Save the transformer you use into mi_transformer
mi_transformer = SelectKBest(mutual_info_regression, k = k).fit(X_train,y_train)
new_feats = X_train.columns[mi_transformer.get_support()]
mi_X_train = pd.DataFrame(mi_transformer.transform(X_train), columns = new_feats)
mi_X_test = pd.DataFrame(mi_transformer.transform(X_test), columns = new_feats)



In [13]:
for feature, importance in zip(house_features.columns, mi_transformer.scores_):
    print(f"The MI score for {feature} is {importance}")

The MI score for MedInc is 0.3996064376238433
The MI score for HouseAge is 0.031907849744349015
The MI score for AveRooms is 0.10789709054525165
The MI score for AveBedrms is 0.0306212721875756
The MI score for Population is 0.024542115170730128
The MI score for AveOccup is 0.07001310248399673
The MI score for Latitude is 0.36536065139287377
The MI score for Longitude is 0.402943613477186


Do these values match what you expected? What are the most important features to use for predicting house prices?

In [14]:
assert mi_transformer.k == k
assert isinstance(mi_transformer, SelectKBest)
assert len(mi_transformer.scores_) == 8
assert mi_X_train.shape == (16512, k)
assert mi_X_test.shape == (4128, k)

Since the focus in this exercise is on the feature learning and not on the supervised learning portion, we will use a simple estimator (linear regression) for the model training portions.

In [15]:
miEst = LinearRegression().fit(mi_X_train, y_train)

In [16]:
print(f"The mean squared error when training on the MI selected features is {mean_squared_error(y_train, miEst.predict(mi_X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, miEst.predict(mi_X_test))}")

The mean squared error when training on the MI selected features is 0.5494954356737959.
When testing on the test data, the mean squared error is 0.5642909967350344


## Wrapper Feature Selection

Now try using [recursive feature elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) to select the 2 features we will use instead.

In [17]:
# Use an RFE object to determine the k features to select from X_train using a step of 2
# Save the rfe object as rfe_transformer
# Create rfe_X_train and rfe_X_test as the updated features based on the RFE output

rfeEst = LinearRegression()
rfe_transformer = RFE(rfeEst, n_features_to_select = k, step = 2).fit(X_train, y_train)
rfe_X_train = rfe_transformer.transform(X_train)
rfe_X_test = rfe_transformer.transform(X_test)



In [18]:
assert isinstance(rfe_transformer, RFE)
assert rfe_transformer.step == 2
assert len(rfe_transformer.support_) == 8
assert rfe_X_train.shape == (16512, k)
assert rfe_X_test.shape == (4128, k)

In [19]:
rfeEst = LinearRegression().fit(rfe_X_train, y_train)

In [20]:
print(f"The mean squared error when training on the RFE selected features is {mean_squared_error(y_train, rfeEst.predict(rfe_X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, rfeEst.predict(rfe_X_test))}")

The mean squared error when training on the RFE selected features is 0.5451672299133382.
When testing on the test data, the mean squared error is 0.5580832992075773


## Embedded Methods

For the embedded methods feature selection example, we will use Lasso. For this task you should use [LassoCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV) and **not** [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) so that it trains with various values for alpha.

Since this is an embedded method, the feature selection will occur directly in the model.

In [21]:
# Create a LassoCV model trained with 10 alphas and save it to lassoClf
lassoClf = LassoCV(n_alphas = 10).fit(X_train, y_train)

In [22]:
lassoClf.coef_

array([ 3.83899283e-01,  1.13979529e-02,  1.56884471e-03,  0.00000000e+00,
       -4.07469366e-07, -4.61025303e-03, -3.24168177e-01, -3.23939467e-01])

In [23]:
lassoClf.alpha_

0.03577400513173515

In [24]:
assert lassoClf
assert isinstance(lassoClf, LassoCV)
assert len(lassoClf.coef_) == 8

In [25]:
print(f"The mean squared error when training using lasso is {mean_squared_error(y_train, lassoClf.predict(X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, lassoClf.predict(X_test))}")

The mean squared error when training using lasso is 0.5447589792844251.
When testing on the test data, the mean squared error is 0.5594157721934994


## Feature Extraction

Here we'll use [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to extract features from the data.

In [26]:
# Select the best k features using PCA
# Save the PCA transformer object as pca_transformer
# Transform X_train and X_test into the new shape and save to pca_X_train and pca_X_test

pca_transformer = PCA(n_components = k).fit(X_train, y_train)
pca_X_train = pca_transformer.transform(X_train)
pca_X_test = pca_transformer.transform(X_test)

In [27]:
assert pca_transformer 
assert isinstance(pca_transformer, PCA)
assert pca_transformer.n_components == k
assert pca_X_train.shape == (16512, k)
assert pca_X_test.shape == (4128, k)

In [28]:
pcaEst = LinearRegression()
pcaEst.fit(pca_X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
print(f"The mean squared error when training using PCA is {mean_squared_error(y_train, pcaEst.predict(pca_X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, pcaEst.predict(pca_X_test))}")

The mean squared error when training using PCA is 1.3225357633272015.
When testing on the test data, the mean squared error is 1.299454026822007


In [30]:
pca_transformer.get_covariance()

array([[ 2.59881878e+00, -2.85404338e+00,  1.69458762e-01,
         2.02649494e-02,  9.88022268e+00,  2.12524850e-02,
         1.77252627e-01, -1.13858541e-01],
       [-2.85404338e+00,  1.58777213e+02, -4.81380579e+00,
        -4.69596878e-01, -4.30539684e+03,  1.01359303e+00,
         4.93684706e-01, -2.87568750e+00],
       [ 1.69458762e-01, -4.81380579e+00,  3.15500331e+00,
         8.25054593e-02, -1.92964965e+02,  7.47416667e-02,
         1.11027923e+00, -9.23849732e-01],
       [ 2.02649494e-02, -4.69596878e-01,  8.25054593e-02,
         2.54365791e+00, -3.40875068e+01, -6.12764836e-03,
         1.60142020e-01, -1.36806830e-01],
       [ 9.88022268e+00, -4.30539684e+03, -1.92964965e+02,
        -3.40875068e+01,  1.30659246e+06,  5.35350485e+02,
        -2.80412604e+02,  2.40965884e+02],
       [ 2.12524850e-02,  1.01359303e+00,  7.47416667e-02,
        -6.12764836e-03,  5.35350485e+02,  4.14906644e+01,
        -7.41569408e-02,  1.75039261e-01],
       [ 1.77252627e-01,  4.936847

Sometimes PCA may perform poorly for regression cases. See [this answer](https://stats.stackexchange.com/a/52798) on StackExchange for more info. In the next lecture, we will cover other methods of feature extraction and show examples where PCA is even more effective.

## Feedback

In [32]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    return "All good!"

In [33]:
print(feedback())

All good!
